In [ ]:
#Install all needed packages
using Pkg;
Pkglist = Pkg.installed()
if !haskey(Pkglist,"HTTP");println("Installing package HTTP");Pkg.add("HTTP");end;
if !haskey(Pkglist,"DelimitedFiles");println("Installing package DelimitedFiles");Pkg.add("DelimitedFiles");end;
if !haskey(Pkglist,"StatsBase");println("Installing package StatsBase");Pkg.add("StatsBase");end;
if !haskey(Pkglist,"Statistics");println("Installing package Statistics");Pkg.add("Statistics");end;
if !haskey(Pkglist,"MLDataUtils");println("Installing package MLDataUtils");Pkg.add("MLDataUtils");end;
if !haskey(Pkglist,"OMGP");println("Installing package OMGP"); Pkg.add(pwd()*"/OMGP"); end;
println("All needed packages installed")

In [ ]:
#Include needed packages
using HTTP, DelimitedFiles, StatsBase, Statistics, MLDataUtils, Random
links = Dict("Electricity"=>"https://www.openml.org/data/get_csv/2419/electricity-normalized.arff",
                "Shuttle"=>"https://www.openml.org/data/get_csv/4965262/shuttle.arff")
dataset="Shuttle"; #Choose between the list above 
data,header = readdlm(HTTP.get(links[dataset]).body,',',header=true);
data

In [ ]:
#Format the data
if dataset == "Electricity"
    global y_1 = data[1,end]; data[:,end] = data[:,end].== y_1
elseif dataset == "Shuttle"
    global y_1 = data[3,end]; 
end

#Convert it into -1,1
data[:,end]=(sign.(data[:,end]*2 .-1)); data = Matrix{Float64}(data); 
nSamples,nDims = size(data);nDims-=1
#Normalize the data
for i in 1:nDims
    data[:,i] = zscore(data[:,i])
end

In [ ]:
#Shuffle the data and split it into train/test
data = data[shuffle(1:nSamples),:];
train, test = Matrix.(splitobs(data, at = 0.7,obsdim=1));
X_train = train[:,1:end-1]; y_train = train[:,end];
X_test = test[:,1:end-1]; y_test = test[:,end];

In [ ]:
using OMGP
#Creating the model and initializing the variables
model = SparseXGPC(X_train,y_train;m=100,Stochastic=true,batchsize=100,
                    kernel=RBFKernel(3.0),Autotuning=true,noise=1e-3,VerboseLevel=2);

In [ ]:
#You can chose to run with a callback function which will measure metrics during the training, it is much slower thouhg
metrics,callback = getLog(model,X_test=X_test,y_test=y_test,iter_points=vcat(1:9,10:10:99,100:100:999,1000:1000:9999))
model.train(iterations=100,callback=callback)
#Or do a normal training (faster)
# model.train(iterations=100)


In [ ]:
y_pred = model.predict(X_test);
println("Accuracy is $(mean(y_pred.==y_test))")

In [ ]:
if @isdefined metrics
    if !haskey(Pkglist,"Plots");println("Installing package Plots"); Pkg.add("Plots"); end;
    using Plots
    plot(plot(metrics[:ELBO],lab="ELBO",title="",xlabel="Iterations"),
        plot(metrics[:test_error],lab="Test Error",title="",xlabel="Iterations"),
        plot(metrics[:mean_neg_loglikelihood],lab="Mean Neg. Log LL",title="",xlabel="Iterations"),
        plot(metrics[:kernel_param],lab="kernel parameter",title="",xlabel="Iterations"))
end